# Temperature response {#sec-temperature}

## Objectives {.unnumbered}

-   Review common temperature response functions used in plant biology
-   Evaluate critically the concept of thermal time known as growing degree day
-   Create a system in `Cropbox` to calculate growing degree days (GDD) and cumulative GDD (cGDD)


## Readings {.unnumbered}

-   (recommended) Johnson IR, Thornley JHM (1985) Temperature dependence of plant and crop process. Annals of Botany 55(1), 1-24 (<https://doi.org/10.1093/oxfordjournals.aob.a086868>)
-   (recommended) Kim S-H, Hsiao J, Kinmonth-Schultz H (2019) Advances and improvements in modeling plant processes. In 'Advances in Crop Modelling for a Sustainable Agriculture.' Ed. K Boote) pp. 3-43. (Burleigh-Dodds: Cambridge, UK) (<https://doi.org/10.19103/AS.2019.0061.01>)


## Questions {.unnumbered}

1. Draw how you think a typical temperature response of biological processes looks like. Label the axes and key features of the curve. And discuss factors that drive the shape of the curve.  
2.  What is growing degree days (GDD) and what may be cons and pros of applying the GDD approach for growh, development, and other processes in plants?

## Introduction

Temperature is one of the most, if not the most, influential environmental variables that affect the biology of organisms and plants are not an exception. Temperature dictates biological processes at all levels from molecular and cellular scales to organismic and ecosystem processes. We will review some of the functions that are commonly used to describe the temperature dependence of biochemical, physiological, phenological, and ecological processes in plants. See @Johnson1985 for a more comprehensive review.

## Thermal time for plant development

Temperature drives plant development. Understanding how temperature affects plant processes is fundamental for modeling plant processes. We hear a lot about phenolgy these days and how climate change is shifting the phenology of plants and animals. [Phenology](https://budburst.org/phenology) is a study of developmental changes in biological organisms. Temperature drives the progression of developmental phases in plants and thus is a major driving variable in most if not all phenology models. The majority of materials covered in this section is available in chapter 1 by Kim et al. in [Boote (2019)](https://doi.org/10.1201/9780429266591). Included in this note is a summary of concepts and contexts. While there are other environmental and physiological factors that influence plant phenology including photoperiod, light integral, water deficit, and hormonal relations, these factors are not explicitly addressed here. See [Johnson and Thonrley (1985)](https://doi.org/10.1093/oxfordjournals.aob.a086868) for more detailed review of temperature dependence of plant processes. 

### Growing Degree Days (GDD)
You might have heard the terms like growing degree days (GDD), thermal units, heat units, heat sums, temperature sums, and thermal-time that are used to relate the rate of plant or insect development to temperature. They are largely synonymous. The concept of thermal-time or thermal-units derives from the long-standing observation and assumption that timing of development is primarily driven by temperature in plants and the relationship is largely linear. The linear relationship is generally held true over normal growing temperatures that are bracketed by the base temperature ($T_b$) and optimal temperature ($T_{opt}$). Many existing crop models and tree growth models use thermal-unit approaches (e.g., GDD) for modeling phenology with some modifications to account for other factors like photoperiod, vernalization, dormancy, and stress. The growing degree days (GDD) is defined as the difference between the average daily air temperature ($T$) and the base temperature below which the developmental process stops. The bigger the difference in a day, the faster the development takes place up to a certain optimal temperature ($T_{opt}$). The  cumulative GDD (cGDD) since the growth initiation (e.g., sowing, imbibition for germination) is then calculated by:

$$
    \mathrm{GDD}(T) = \max \{ 0, \min \{ T, T_{opt} \} - T_b \},
$$ {#eq-GDD}

$$
    \mathrm{cGDD} = \sum_i^n { \mathrm{GDD}(T_i)} = \sum_i^n {\max \{ 0, \min \{ T_i, T_{opt} \} - T_b \}},
$$ {#eq-cGDD}

where $T_i$ is the average daily temperature on day $i$ and $n$ is the number of days since the growth initiation. The base temperature ($T_b$) and optimal temperature ($T_{opt}$) are species-specific and sometimes process-specific (e.g., different for vegetative growth and reproductive development). The GDD approach assumes that the rate of development increases linearly with temperature up to $T_{opt}$ beyond which the rate remains constant.
The GDD approach is simple and effective in most situations. However, it often fails to represent developmental responses that are nonlinear as shown in very low or high temperatures that are stressful for plants. See the section on nonlinear thermal-time functions below.

Recently, a similar concept called Killing Degree Days (KDD) was coined to describe the damaging effects of extreme heats on crops ([Butler and Huybers, 2015](https://doi.org/10.1088/1748-9326/10/3/034009)). KDD accumulated when the daily maximum temperature ($T_{max}$) exceeded $T_{opt}$.
  
$$
\mathrm{KDD}(T_{max})= \min\{0, T_{max}-T_{opt}\} 
$${#eq-KDD}
$$
\mathrm{cKDD} = \sum_i^n { \mathrm{KDD}(T_{max_i})} = \sum_i^n {\min\{0, T_{max_i}-T_{opt}\}} 
$$ {#eq-cKDD}


where $T_{max_i}$ is the daily maximum temperature on day $i$ and $n$ is the number of days since the growth initiation. The cKDD can be used as an index to quantify the impact of extreme heat events on crop yield.


Using `Cropbox` let's create a system to calculate GDD and cGDD.

In [ ]:
using Cropbox, Plots

In [ ]:
"Growing degree days and killing degree days calculator"
@system DegreeDays begin
    T:  temperature                              ~ preserve(parameter, u"°C")
    Tb: base_temperature                         ~ preserve(parameter, u"°C")
    To: optimal_temperature                      ~ preserve(parameter, u"°C")
    GDD_rating: maturity_growing_degree_rating   ~ preserve(parameter, u"K")
    
    GD(T, Tb, To): growing_degree => begin
        min(T, To) - Tb
    end ~ track(min = 0, u"K")

    GDD(GD): growing_degree_day => begin
        GD / 1u"d"
    end ~ track(u"K/d")

    cGDD(GDD): cumulative_growing_degree_day ~ accumulate(u"K")

    KD(T, To): killing_degree => begin
        T - To
    end ~ track(min = 0, u"K")

    KDD(KD): killing_degree_day => begin
        KD / 1u"d"
    end ~ track(u"K/d")

    cKDD(KDD): cumulative_killing_degree_day ~ accumulate(u"K")
end

In [ ]:
@system DDController(DegreeDays, Controller)

In [ ]:
# prepare a configuration to simulate and visualize
dd_config = @config (
    Clock => (;
        step = 1u"d",
    ),
    DegreeDays => (;
        Tb = 10.0,
        To = 30.0,
        GDD_rating = 1500,
    ),
)

Let's create a plot showing how GDD relates to mean daily temperatures (T). Try changing parameter values in the configuration and replot. How would KDD behave if added to the plot?

In [ ]:
visualize(DDController, :T, [:GDD];
    config = dd_config,
    xstep = :0 => :T => 0:40,
    kind = :line,
);

### Nonlinear thermal-time functions

The thermal-unit approaches like GDD are simple and effective in most situations. However, they often fail to represent developmental responses that are nonlinear as shown in very low or high temperatures that are stressful for plants. In fact, like all other biological processes, the relationship between plant processes and temperature is nonlinear or curvilinear with initial quasi-exponential to linear increase reaching a peak after which the rate declines rapidly. 

Different mathematical functions such as the **beta distribution function** [@Yan1999] have been increasingly used to describe the rate of plant development. Although the beta function and other similar nonlinear models remain empirical like the thermal-unit approaches, they are often more flexible than the linear GDD approach, and their parameters based on cardinal temperatures ($T_{opt}$, and $T_{ceil}$; $T_{ceil}$ is the ceiling temperature above which the developmental process ceases) are biologically meaningful, easily interpretable, and straightforward to determine from calibration.

$$
    r(T) = R_{max} \left( \frac{T_{ceil} - T}{T_{ceil} - T_{opt}} \right) \left( \frac{T}{T_{opt}} \right) ^ {\frac{T_{opt}}{T_{ceil} - T_{opt}}}
$${#eq-beta}

In [ ]:
@system BetaFunc begin
    T:  temperature              ~ preserve(parameter, u"°C")
    Tn: minimum_temperature => 0 ~ preserve(parameter, u"°C")
    To: optimal_temperature      ~ preserve(parameter, u"°C")
    Tc: ceiling_temperature      ~ preserve(parameter, u"°C")
    Rx: maximum_rate        => 1 ~ preserve(parameter, u"d^-1")

    r(T, Tn, To, Tc, Rx): rate => begin
        Rx * (Tc - T) / (Tc - To) * ((T - Tn) / (To - Tn))^((To - Tn) / (Tc - To))
    end ~ track(min = 0, max = Rx, u"d^-1")

    R(r): cumulative_rate ~ accumulate
end

In [ ]:
@system BFController(BetaFunc, Controller)
bf_config = @config (
    BetaFunc => (;
        Rx = 0.58,
        To = 31.4,
        Tc = 41.0,
    ),
)

In [ ]:
visualize(BFController, :T, :r;
    config = bf_config,
    xstep = :0 => :T => 0:40,
    kind = :line,
);

## Temperature response functions
### The Arrhenius equation

A widely used function to describle the temperature dependence of the rate of biochemical reactions (e.g., photosynthetic carboxylation) catalyzed by an enzyme (e.g., Rubisco) is the Arrhenius equation. There are many different forms of this equation. Two of the commonly used forms are introduced here from [@Yun2020; @Kim2003].

$$
k(T_k, E_a) = \exp\!\left[ \frac{E_a ( T_k - T_{b_k} )}{R \cdot T_k \cdot T_{b_k}} \right]
$${#eq-ArrK}

$E_a$ is the activation energy (kJ mol^−1^) which varies by process (e.g., 66.4 kJ mol^−1^ for mitocondrial respiration; see [@Kim2003]), $T_k$ is the tissue or organ temperature in Kelvin (K) and $T_{b_k}$ is the base temperature assumed to be 298K (i.e., 25 °C ). $R$ is the universal gas constant (8.314 J K mol^−−1^). This temperature response is normalized to be unity at $T_{b_k}$ and scales with temperature to be multiplied by the reaction rate at the reference temperature ($k_b$ or $k_{25}$).

This equation can be re-written in terms of temperature ($T$) in °C as follows [@Kim2003]:

$$
k(T, E_a) = \exp\!\left[\frac{E_a(T-25)}{298 \cdot R \cdot(T+273)}\right]
$$ {#eq-ArrC}


### The $\mathrm{Q_{10}}$ function

Another widely used approach for describing temperature response is the $Q_{10}$ function. This is an empirical approach based on the observation that the rate of a reaction increases by a constant factor over temperature [@Johnson1985]. For example, the $Q_{10}$ value of 2.0 means the reaction rates doubles every 10 K (i.e., 10 °C) increase in temperature. The relationship can be written as:

$$
k(T, Q_{10})= Q_{10}^{\frac{T - T_b}{10}}
$${#eq-q10}

$T_b$ is the base temperature at which the response is normalized to be 1.0 and $Q_{10}$ is the rate constant by which the reaction rate increases for a 10 K temperature increment.

### Functions with a temperature optimum

The approaches introduced so far assume the rate at which a process or reaction occurs increase exponentially with temperature but most biochemical reactions and biological processes have a temperature optimum beyond which the rate decreases rapidly. The temperature dependence of a process with a temperature optimum can be modeled using a peaked Arrhenius function [@Medlyn2002; @Kim2003].

$$
k(T_k, E_a, H, S) = k(T_k, E_a) \cdot \left( 1 + \exp\!\left[ \frac{S \cdot T_{b_k} - H}{R \cdot T_{b_k}} \right] \right) \left( 1 + \exp\!\left[ \frac{S \cdot T_k - H}{R \cdot T_k} \right] \right)^{-1}
$${#eq-Arr-peak}

$E_a$ and $R$ are as defined above. $H$ is the curvature parameter determining the rate of decrease above the peak temperature (kJ mol^−1^) and $S$ is the entropy factor (J mol^−1^ K). An alternative form of this equation with the optimum temperature ($T_{opt}$) as a parameter can be found in [@Medlyn2002]. Other empirical models with a temperature optimum (e.g., beta function) exist and have been used to describe the temperature response of plant development and other processes in plant and crop modeling [@Kim2007; @Yan1999; @Johnson1985]

## Homework Problems
1. Compare the temperature response of a process using the Arrhenius equation (@eq-ArrC) and the $Q_{10}$ function (@eq-q10). Assume an activation energy ($E_a$) of 60 kJ mol^−1^ for the Arrhenius equation and a $Q_{10}$ value of 2.0. Plot the rate of the process over a temperature range of 0 °C to 40 °C for both functions. How do the two functions differ in their temperature response?
